<a href="https://colab.research.google.com/github/0x1beef/uap/blob/main/src/transmittance_lowtran.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/0x1beef/uap/blob/main/src/transmittance_lowtran.ipynb">
    <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/>
</a>

# Transmittance

The following will investigate the degree to which the atmosphere absorbs/scatters infrared radiation in various scenarios relevant to UAP cases.

## Install LOWTRAN and its dependencies

The most comprehensive atmospheric transmittance models like MODTRAN are not freely available. For now we'll test an older lower resolution model, LOWTRAN. The latter is written in Fortran but it has a python wrapper.

The following code grew out of [examples](https://github.com/space-physics/lowtran/blob/main/example/HorizontalTransmittance.py) that are available with in its [Github repository](github.com/space-physics/lowtran). The repository includes the Fortran source file [here](https://github.com/space-physics/lowtran/tree/main/src/lowtran/fortran) as well. A more detailed user guide for LOWTRAN can also be downloaded from [here](https://www.researchgate.net/publication/235203858_User_guide_to_LOWTRAN_7).

It would also be worth investigating [libradtran](https://www.libradtran.org/doku.php?id=start) and its wrapper [libradtranpy](https://github.com/LSSTDESC/libradtranpy). Its latest release is only a year old, and is advertised as being more accurate.

In [ ]:
!pip install lowtran meson ninja

## Code for calculating and plotting transmittance data

An interface to the LOWTRAN library is created that is more convenient for our purposes.

In [ ]:
import matplotlib.pyplot as plt
import lowtran
import math

def feet_to_km(feet):
  return feet * 0.0003048
def km_to_feet(feet):
  return feet / 0.0003048
def nm_to_km(nm):
  return nm / 1.852
def km_to_nm(nm):
  return nm * 1.852

def slant_range(alt_1, alt_2, horiz_range):
  delta_alt = abs(alt_1 - alt_2)
  return math.sqrt(horiz_range**2 + delta_alt**2)

def calc(alt_1_kft = 0, alt_2_kft = None, model = 6, dist_nm = None):
  if dist_nm is None:
    dist = 1.0
  else:
    dist = nm_to_km(dist_nm)

  alt_1 = feet_to_km(alt_1_kft * 1000)
  if alt_2_kft is None:
    alt_2 = alt_1
    itype = 1 # horizontal path
  else:
    alt_2 = feet_to_km(alt_2_kft * 1000)
    itype = 3 # slant path
    dist = slant_range(alt_1, alt_2, dist)

  c1 = {
    'zmdl': 0, # atmosphere starts from ground level
    'h1': alt_1,
    'h2': alt_2,
    'range_km': dist,
    'wlshort': 3000,
    'wllong': 5000,
    'wlstep': 20,
    # based on lowtran.horiztrans:
    'model': model,  # 5 = subartic winter
    'itype': itype,
    'iemsct': 0,  # 0: transmittance model
    'im': 0,  # 1: for user-defined atmosphere on horizontal path (see Lowtran manual p.42)
    'ird1': 0,  # 1: use card 2C2
  }

  TR = lowtran.golowtran(c1) #.squeeze()
  return (TR, c1)

def trunc(nr):
  return f'{nr:.2f}'.rstrip('0').rstrip('.')

# based on lowtran.plot.horiz
def plot(trans, c1, horiz_range_km = None, to_feet_nm = False):
  title = 'Transmittance\n'

  rng = c1['range_km'] if horiz_range_km is None else horiz_range_km

  rng_units = 'nm' if to_feet_nm else 'km'
  alt_units = 'k ft' if to_feet_nm else ' km'
  rng = km_to_nm(rng) if to_feet_nm else rng
  h1 = km_to_feet(c1['h1']) / 1000.0 if to_feet_nm else c1['h1']
  h2 = km_to_feet(c1['h2']) / 1000.0 if to_feet_nm else c1['h2']

  if c1['itype'] == 1 or horiz_range_km is not None:
    title += f'Horizontal range {trunc(rng)} {rng_units}'
  else:
    title += f'Slant range {trunc(rng)} {rng_units}'

  if h1 == h2:
    title += f' @ {trunc(h1)}{alt_units} altitude\n'
  else:
    title += f' @ {trunc(h1)}{alt_units} to {trunc(h2)}{alt_units} altitude\n'

  model_names = [ 'Tropical', 'Midlatitude summer', 'Midlatitude winter',
    'Subarctic summer', 'Subarctic winter', 'U.S. standard' ]
  if c1['model'] == 0:
    title += f'User defined atmosphere: pressure: {c1['p']} mbar, temperature {c1['t']} K'
  elif c1['model'] >= 1 and c1['model'] <= len(model_names):
    title += model_names[c1['model'] - 1] + ' atmosphere'

  ax = plt.figure().gca()

  ax.plot(trans.wavelength_nm, trans['transmission'].squeeze())

  ax.set_xlabel('wavelength [nm]')
  ax.set_ylabel('transmission (unitless)')
  ax.set_title(title)
  ax.set_ylim(0, 1)
  # ax.legend(loc='best')
  ax.grid(True)
  ax.autoscale(True, axis='x', tight=True)

def calc_plot(alt_1_kft = 0, alt_2_kft = None, model = 6, dist_nm = None):
  (TR, c1) = calc(alt_1_kft, alt_2_kft, model, dist_nm)
  if dist_nm is None:
    plot(TR, c1, to_feet_nm = False)
  else:
    plot(TR, c1, horiz_range_km = nm_to_km(dist_nm), to_feet_nm = True)
  plt.show()

## Reproducing the Standard Transmittance Graph

The transmittance of various wavelengths in the infrared spectrum through the atmosphere is typically shown like this:

![transmittance](https://upload.wikimedia.org/wikipedia/commons/1/1c/Atmosfaerisk_spredning.png)

The transmittance is shown over 1km horizontally at sea level. We can reproduce the part of that graph that's relevant to the ATFLIR's InSb sensor, 3-5 μm:

In [ ]:
calc_plot()

## The Standard setup at Altitude

The only thing we change here is the altitude to see what a dramatic effect it has, with some values getting close to 100%.

In [ ]:
calc_plot(alt_1_kft=25)

## Transmittance for the Gimbal Distant Jet scenario

300 nm off the coast of Jacksonville in Jan 2015 the midlatitude winter atmosphere should be most relevant, with an atmosphere that is more dry than during the summer according to [this](http://modtran.spectral.com/static/modtran6/html/help_atmosphere_model.html?v=3).

We can see that a signifcant portion of the IR a distant jet would emit is still transmitted through the atmosphere at this distance.

In [ ]:
calc_plot(alt_1_kft = 25, alt_2_kft = 19, model = 3, dist_nm = 30)

## Transmittance for the clouds at various distances

In Sitrec the clouds in Gimbal are [currently](https://github.com/MickWest/Sitrec2/blob/abb7c7a2e7d690109e5fbcb060ba20437c31582e/src/JetStuff.js#L1066C16-L1066C21) at 11740 feet in altitude.

A surprising result is that the transmittance graph seems the same whether the distance to the clouds is 40 or 100 nm. Further investigation is needed to see whether this could indeed be real.

In [ ]:
calc_plot(alt_1_kft = 25, alt_2_kft = 11.7, model = 3, dist_nm = 100)
calc_plot(alt_1_kft = 25, alt_2_kft = 11.7, model = 3, dist_nm = 40)

## Chilean Navy UAP scenario

I couldn't find a model where the jet could be seen 90 nm away here. Maybe none of them account for just how dry that lower atmosphere can get in Chile, or maybe some other settings are needed.

In [ ]:
calc_plot(alt_1_kft = 4.5, alt_2_kft = 38, model = 3, dist_nm = 90)

## Western United States UAP scenario

An [AARO case resolution report](https://www.aaro.mil/Portals/136/PDFs/case_resolution_reports/Case_Resolution_of%20_Western_United_States_Uap_508-02262024.pdf) for a "Western United States UAP" involves a jet seen 300 nm away. The report doesn't mention the altitude of the observing platform. This [post](https://www.metabunk.org/threads/dave-falchs-claims-about-ir-visibility-in-the-chilean-navy-ufo-ib6830.13664/post-323818) found that with refraction it might've been as low as 50k feet. It doesn't say whether it's winter or summer of 2021. We'll just assume it was summer.

The following suggests those jets should be visible.

In [ ]:
calc_plot(alt_1_kft = 50, model = 2, dist_nm = 300)